In [141]:
stn_pairs = []
avg_speeds = {
    "North-South": 41,
    "East-West": 43,
    "Circle": 36,
    "Thomson-East Coast": 40,
    "North East": 34,
    "Downtown": 38,
}
stn_neighbors = {}

In [142]:
def load_data():
    file_path = "MRT.txt"
    file = open(file_path, "r")
    lines = [line.strip() for line in file.readlines() if line.strip()]
    data_lines = lines[2:]
    file.close()

    for data_line in data_lines:
        data_list = data_line.split(";")
        stn_pair = {}
        stn_pair["from_stn"], stn_pair["to_stn"] = [
            i.split(" ", 1)[1] for i in data_list[1].split(" <-> ")
        ]
        stn_pair["line"] = data_list[2]
        stn_pair["distance"] = float(data_list[3])
        stn_pair["duration"] = stn_pair["distance"] / avg_speeds[stn_pair["line"]]
        (
            stn_pair["first"],
            stn_pair["last"],
            stn_pair["first_opp"],
            stn_pair["last_opp"],
        ) = data_list[4:8]
        stn_pairs.append(stn_pair)

        for stn_pair in stn_pairs:
            from_stn, to_stn, distance, duration, line = (
                stn_pair["from_stn"],
                stn_pair["to_stn"],
                stn_pair["distance"],
                stn_pair["duration"],
                stn_pair["line"],
            )
            for start, end in (from_stn, to_stn), (to_stn, from_stn):
                stn_neighbors.setdefault(start, {})
                if (
                    end not in stn_neighbors[start]
                    or duration < stn_neighbors[start][end]["duration"]
                ):
                    stn_neighbors[start][end] = {
                        "distance": distance,
                        "duration": duration,
                        "line": line,
                    }

In [143]:
load_data()

In [144]:
def bellman_ford(graph, start, end):
    distances = {station: float("inf") for station in graph}
    predecessors = {station: None for station in graph}
    distances[start] = 0

    for _ in range(len(graph) - 1):  # Relax edges |V|-1 times
        for u in graph:
            for v in graph[u]:
                weight = graph[u][v]["duration"]
                if distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight
                    predecessors[v] = u

    # Reconstruct path
    path = []
    current = end
    while current is not None:
        path.append(current)
        current = predecessors[current]
    return path[::-1] if path[-1] == start else None  # Reverse path

print(bellman_ford(stn_neighbors, "Pioneer", "Changi Airport"))  # Output: Shortest path based on duration

['Pioneer', 'Boon Lay', 'Lakeside', 'Chinese Garden', 'Jurong East', 'Clementi', 'Dover', 'Buona Vista', 'Holland Village', 'Farrer Road', 'Botanic Gardens', 'Stevens', 'Newton', 'Little India', 'Rochor', 'Bugis', 'Lavender', 'Kallang', 'Aljunied', 'Paya Lebar', 'Eunos', 'Kembangan', 'Bedok', 'Tanah Merah', 'Expo', 'Changi Airport']
